In [202]:
import ipywidgets as ipw
import utils
import datetime
DATA_MODEL = utils.read_yaml("/home/jovyan/aiida-openbis/Notebooks/Metadata_Schemas_LinkML/materialMLinfo.yaml")

In [203]:
class ObjectPropertiesWidgets(ipw.VBox):
    def __init__(self, task):
        super().__init__()
        self.properties_widgets = {}
        self.all_schema_classes = DATA_MODEL["classes"]
        self.all_schema_slots = DATA_MODEL["slots"]
        object = self.all_schema_classes[task]
        object = self.get_properties_recursive(object)
        
        for property in object["slots"]:
            property_widget = self.get_property_widget(property)
            if property_widget:
                self.properties_widgets[property] = property_widget
        
        self.children = list(self.properties_widgets.values())
    
    def get_property_widget(self, property):
        property_widget = None
        property_settings = self.all_schema_slots[property]
        property_multivalued = property_settings["multivalued"]
        property_description = property_settings["description"]
        property_range = property_settings["range"]
        property_openbis_type = property_settings["annotations"]["openbis_type"]
        
        # Multivalued properties are a special case of properties
        if property_multivalued:
            return
        
        if property_openbis_type == "VARCHAR":
            property_widget = utils.Text(
                description = property_description, 
                layout = ipw.Layout(width = "400px"), 
                placeholder = "", 
                style = {'description_width': "110px"}
            )
        
        elif property_openbis_type == "MULTILINE_VARCHAR":
            property_widget = utils.Textarea(
                description = property_description, 
                layout = ipw.Layout(width = "400px", height = "100px"), 
                placeholder = "", 
                style = {'description_width': "110px"}
            )
        
        elif property_openbis_type == "BOOLEAN":
            property_widget = utils.Checkbox(
                description = property_description, 
                layout = ipw.Layout(width = "400px"), 
                style = {'description_width': "110px"},
                indent = True, value = False
            )
        
        elif property_openbis_type == "DATE":
            property_widget = ipw.DatePicker(
                description = property_description, 
                layout = ipw.Layout(width = "300px"), 
                style = {'description_width': "110px"},
                value = datetime.date.today()
            )
        
        elif property_openbis_type == "TIMESTAMP":
            property_widget = utils.Text(
                description = property_description, 
                layout = ipw.Layout(width = "400px"), 
                placeholder = "", 
                style = {'description_width': "110px"}
            )
        
        elif property_openbis_type == "INTEGER":
            property_widget = utils.IntText(
                description = property_description, 
                layout = ipw.Layout(width = "200px"), 
                style = {'description_width': "110px"}
            )
        
        elif property_openbis_type == "REAL":
            property_widget = utils.FloatText(
                description = property_description, 
                layout = ipw.Layout(width = "200px"), 
                style = {'description_width': "110px"}
            )

        elif property_openbis_type == "CONTROLLEDVOCABULARY":
            property_vocabulary = DATA_MODEL["enums"][property_range]["permissible_values"].keys()
            property_vocabulary = list(property_vocabulary)
            property_widget = utils.Dropdown(
                description = property_description, 
                layout = ipw.Layout(width = "200px"), 
                style = {'description_width': "110px"},
                options = property_vocabulary,
                value = property_vocabulary[0]
            )
        
        elif property_openbis_type == "JSON":
            property_widget = self.get_property_widgets_recursive(property_description, property_range)
        
        return property_widget
    
    def get_properties_recursive(self, object):
        # Get all the property types until the last parent class
        object_copy = object
        while "is_a" in object_copy:
            parent_object = self.all_schema_classes[object_copy["is_a"]]
            # Used when the object inherits all the properties from another object
            if object["slots"]:
                object["slots"] = parent_object["slots"] + object["slots"]
            else:
                object["slots"] = parent_object["slots"]
                
            object_copy = parent_object
        
        return object

    def get_property_widgets_recursive(self, property, property_range):
        property_widget = []
        if property_range in self.all_schema_classes:
            widget_group_label = ipw.Label(value = property, layout=ipw.Layout(margin='0 0 5px 0', font_weight='bold'))
            property_widget.append(widget_group_label)
            for slot in self.all_schema_classes[property_range]["slots"]:
                if self.all_schema_slots[slot]["annotations"]["openbis_type"] != "Not used":
                    property_widget.append(self.get_property_widget(slot))

        print(property_widget)
        return ipw.VBox(property_widget)

# print(ObjectPropertiesWidgets("Molecule").properties_widgets)
display(ObjectPropertiesWidgets("CrystalConcept"))

# print(ObjectPropertiesWidgets("Deposition").properties_widgets)
# display(ObjectPropertiesWidgets("Deposition"))

ObjectPropertiesWidgets(children=(Text(value='', description='Name', layout=Layout(width='400px'), placeholder…

In [204]:
# import ipywidgets as widgets
# from IPython.display import display

# # Title Label
# title_label = widgets.Label(value="Property Name", 
#                             layout=widgets.Layout(margin='0 0 5px 0', font_weight='bold'))

# # Value and Unit widgets
# value_label = widgets.Label(value="Value:")
# value_input = widgets.Text(placeholder='Enter value')

# unit_label = widgets.Label(value="Unit:")
# unit_input = widgets.Dropdown(options=['kg', 'm', 's'], value='kg')

# # Horizontal layout for Value and Unit
# value_unit_box = widgets.HBox([value_label, value_input, unit_label, unit_input],
#                               layout=widgets.Layout(gap='10px'))

# # Main container with border styling
# main_box = widgets.VBox([title_label, value_unit_box],
#                         layout=widgets.Layout(border='1px solid gray',
#                                               padding='10px',
#                                               width='300px'))

# # Display the box
# display(main_box)